In [2]:
# python standard libraries
import sys
import copy
from pathlib import Path
from datetime import datetime


# PyQt stuff
from PyQt5.QtCore import QTimer, QMutex, QThread
from PyQt5.QtWidgets import QApplication, QDialog, QMainWindow


# import PyQT UIs (converted from .ui to .py)
from ui.UI_mainframe_v3 import Ui_MainWindow
from ui.UI_dialogs import Ui_Dialog


# import my own libs
from libs.PRINT_win_dialogs import strdDialog
from libs.PRINT_data_utilities import *
from libs.PRINT_threads import ReceiveWorker,SendQueueWorker

In [4]:
class Mainframe(QMainWindow, Ui_MainWindow):
    """ main UI of PRINT_py (further details pending) """
    


    #####################################################################################################
    #                                           SETUP                                                   #
    #####################################################################################################
    
    def __init__(self,parent=None):
        super().__init__(parent)
        
        self.setupUi(self)
        self.logEntry('GNRL','GUI running, connecting mainsignals...')
        self.connectMainSignals()           
        
        self.logEntry('GNRL','init threading...')
        self.connectThreads()
        
        self.logEntry('GNRL','connect to Robot...') 
        res = self.connectTCP(1)
        if(res == False):
            self.logEntry('GNRL','failed, exiting...')
            QTimer.singleShot(0, self.close)            # suicide after the actual .exec is finished, exit without chrash

        else:
            self.TCP_ROB_indi_connected.setStyleSheet("border-radius: 20px; \
                                                       background-color: #00aaff;")
            self.receiveThread.start()
            self.setWatchdog()

            self.logEntry('GNRL','setup finished.')



    def connectMainSignals(self):
        self.DC_btt_xPlus.pressed.connect( lambda: self.sendDCCommand('X','+') )
        self.DC_btt_xMinus.pressed.connect( lambda: self.sendDCCommand('X','-') )
        self.DC_btt_yPlus.pressed.connect( lambda: self.sendDCCommand('Y','+') )
        self.DC_btt_yMinus.pressed.connect( lambda: self.sendDCCommand('Y','-') )
        self.DC_btt_zPlus.pressed.connect( lambda: self.sendDCCommand('Z','+') )
        self.DC_btt_zMinus.pressed.connect( lambda: self.sendDCCommand('Z','-') )
        self.DC_btt_extPlus.pressed.connect( lambda: self.sendDCCommand('EXT','+') )
        self.DC_btt_extMinus.pressed.connect( lambda: self.sendDCCommand('EXT','-') )
        self.DC_btt_xyzZero.pressed.connect( lambda: self.setZero([1,2,3]) )
        self.DC_btt_extZero.pressed.connect( lambda: self.setZero([8]) )
        self.DC_btt_home.pressed.connect(self.homeCommand)

        self.NC_btt_xyzSend.pressed.connect( lambda: self.sendNCCommand([1,2,3]) )
        self.NC_btt_xyzExtSend.pressed.connect( lambda: self.sendNCCommand([1,2,3,8]) )
        self.NC_btt_orientSend.pressed.connect( lambda: self.sendNCCommand([4,5,6]) )
        self.NC_btt_orientZero.pressed.connect( lambda: self.setZero([4,5,6]) )

        self.IO_btt_newFile.pressed.connect( lambda: strdDialog("works",Ui_Dialog) )
        self.IO_btt_xyzZero.pressed.connect( lambda: self.setZero([1,2,3]) )
        self.IO_btt_extZero.pressed.connect( lambda: self.setZero([8]) )

        self.SCTRL_btt_forcedStop.pressed.connect( lambda: self.sysCommand('S') )
        self.SGLC_btt_gcodeSglComm.pressed.connect(self.addGcodeSgl)
        self.SGLC_btt_rapidSglComm.pressed.connect(self.addRapidSgl)
        self.SGLC_btt_gcodeSglComm_addByID.pressed.connect( lambda: self.addGcodeSgl(atID = True) )
        self.SGLC_btt_rapidSglComm_addByID.pressed.connect( lambda: self.addRapidSgl(atID = True) )
        # self.action_Exit.triggered.connect(self.close)



    def connectTCP(self,TCPslot = 0):
        """slot-wise connection management, mostly to shrink code length, maybe more functionality later"""

        match TCPslot:
            case 1:
                res,conn = ROBO_tcpip.connect()
            case 2:
                res,conn = PUMP1_tcpip.connect()
            case 3:
                res,conn = PUMP2_tcpip.connect()
        
        if (res == True):
            text = 'connected to ' + str(conn[0]) + " at " + str(conn[1]) + "."
            self.logEntry('CONN',text)
            return True
        elif (res == TimeoutError):
            text = 'timed out while trying to connect ' + str(conn[0]) + " at " + str(conn[1]) + "."
            self.logEntry('CONN',text)
            return False
        elif (res == ConnectionRefusedError):
            text = 'server ' + str(conn[0]) + " at " + str(conn[1]) + " refused the connection."
            self.logEntry('CONN',text)
            return False
        else:
            text = 'connection to ' + str(conn[0]) + " at " + str(conn[1]) + " failed!"
            self.logEntry('CONN',text)
            return False
        



    #####################################################################################################
    #                                          THREADS                                                  #
    #####################################################################################################

    def connectThreads(self):
        """load all threads from PRINT_threads and set signal-slot-connections"""

        self.receiveThread = QThread()
        self.receiveWorker = ReceiveWorker()
        self.receiveWorker.moveToThread(self.receiveThread)
        self.receiveThread.started.connect(self.receiveWorker.run)
        self.receiveThread.finished.connect(self.receiveWorker.stop)
        self.receiveThread.finished.connect(self.receiveWorker.deleteLater)
        self.receiveWorker.logError.connect(self.logEntry)
        self.receiveWorker.dataUpdated.connect( lambda: self.resetWatchdog(1) )
        self.receiveWorker.dataUpdated.connect(self.posUpdate)

        
        self.sendQueueThread = QThread()
        self.sendQueueWorker = SendQueueWorker()
        self.sendQueueWorker.moveToThread(self.sendQueueThread)
        self.sendQueueThread.started.connect(self.sendQueueWorker.run)
        self.sendQueueThread.finished.connect(self.sendQueueWorker.stop)
        self.sendQueueThread.finished.connect(self.sendQueueWorker.deleteLater)
        self.sendQueueWorker.queueEmtpy.connect(self.stopSCTRLQueue)
        self.sendQueueWorker.elemSend.connect(self.sendCommand)
        self.sendQueueWorker.elemSend.connect(self.labelUpdate_onSend)
        self.SCTRL_btt_startQProcessing.pressed.connect( self.startSCTRLQueue )
        self.SCTRL_btt_holdQProcessing.pressed.connect( self.stopSCTRLQueue )
    


    def posUpdate(self,rawDataString,pos,toolSpeed,robo_comm_id):
        global STT_pos
        global STT_toolSpeed
        global STT_robo_comm_id

        mutex.lock()
        STT_pos = pos
        STT_toolSpeed = toolSpeed
        STT_robo_comm_id = robo_comm_id
        mutex.unlock()

        self.labelUpdate_onReceive(rawDataString)





    #####################################################################################################
    #                                          WATCHDOGS                                                #
    #####################################################################################################

    def setWatchdog(self):
        """ set Watchdog, just one to check the robots TCP connection for now (receiveWD) at least every 10 sec """

        self.receiveWD = QTimer()
        self.receiveWD.setSingleShot(True)
        self.receiveWD.setInterval(10000)
        self.receiveWD.timeout.connect(lambda: self.watchdogBite(1))
        self.receiveWD.start()
        self.logEntry('WDOG','Watchdog 1 (receiveWD) started.')
        


    def resetWatchdog(self, dognumber=0):
        """ reset the Watchdogs, receiveWD on every newly received data block """

        match dognumber:
            case 1:
                self.receiveWD.start()
            case _:
                self.logEntry('WDOG','Watchdog reset failed, invalid dog number given')



    def watchdogBite(self, dognumber=0):
        """ close the UI on any biting WD, log info """

        match dognumber:
            case 1: 
                self.logEntry('WDOG','Watchdog 1 (receiveWD) has bitten!')
            case _:
                self.logEntry('WDOG','Watchdog (unidentified) has bitten!')
        self.close()



    #####################################################################################################
    #                                        LOG FUNCTION                                               #
    #####################################################################################################

    def logEntry(self, source='no source', text=''):
        """ set one-line for log entries, safes A LOT of code """

        text = str(datetime.now().strftime('%Y-%m-%d_%H%M%S ')) + source + ": " + text + "\n"
        self.SET_disp_logEntry.setText(text)

        try:
            logfile = open(logpath,'a')
        except FileExistsError:
            pass

        logfile.write(text)
        logfile.close()



    #####################################################################################################
    #                                        QLABEL UPDATES                                             #
    #####################################################################################################

    def labelUpdate_onReceive(self,dataString):
        """ update all QLabels in the UI that may change with newly received data from robot """

        self.TCP_ROB_disp_readBuffer.setText(dataString)  

        self.SCTRL_disp_robCommID.setText(str(STT_robo_comm_id))
        self.SCTRL_disp_progCommID.setText(str(SC_curr_comm_id))
        try:
            self.SCTRL_disp_buffComms.setText(str(SC_queue[0].ID - STT_robo_comm_id))
        except TypeError or IndexError:
            self.SCTRL_disp_buffComms.setText(str(SC_curr_comm_id - STT_robo_comm_id))
        self.SCTRL_disp_elemInQ.setText( str(len(SC_queue)) )                              

        self.DC_disp_x.setText(str(STT_pos.X))
        self.DC_disp_y.setText(str(STT_pos.Y))
        self.DC_disp_z.setText(str(STT_pos.Z))
        self.DC_disp_ext.setText(str(STT_pos.EXT))

        self.NC_disp_x.setText(str(STT_pos.X))
        self.NC_disp_y.setText(str(STT_pos.Y))
        self.NC_disp_z.setText(str(STT_pos.Z))
        self.NC_disp_xOrient.setText(str(STT_pos.X_ori))
        self.NC_disp_yOrient.setText(str(STT_pos.Y_ori))
        self.NC_disp_zOrient.setText(str(STT_pos.Z_ori))
        self.NC_disp_ext.setText(str(STT_pos.EXT))

        self.TERM_disp_tcpSpeed.setText(str(STT_toolSpeed))
        self.TERM_disp_robCommID.setText(str(STT_robo_comm_id))
        self.TERM_disp_progCommID.setText(str(SC_curr_comm_id))

        

    def labelUpdate_onSend(self,entry):
        """ update all UI QLabels that may change when data was send to robot"""

        entries = SC_queue.display()
        self.SCTRL_arr_queue.clear()
        self.SCTRL_arr_queue.addItems(entries)
        self.SCTRL_disp_elemInQ.setText( str(len(SC_queue)) )
        try:
            self.SCTRL_disp_buffComms.setText(str(SC_queue.queue[0].ID - STT_robo_comm_id))
        except IndexError:
            pass



    def labelUpdate_onQueueChange(self):
        """ show when new entries have been successfully placed in Queue """

        entries = SC_queue.display()
        self.SCTRL_arr_queue.clear()
        self.SCTRL_arr_queue.addItems(entries)
    


    #####################################################################################################
    #                                         COMMAND QUEUE                                             #
    #####################################################################################################


    def startSCTRLQueue(self):
        """ set UI indicators, send the boring work of timing the command to our trusty threads """

        self.sendQueueThread.start()
        css = "border-radius: 20px; \
               background-color: #00aaff;"
        self.SCTRL_indi_qProcessing.setStyleSheet(css)
        self.DC_indi_robotMoving.setStyleSheet(css)
        self.TCP_indi_qProcessing.setStyleSheet(css)
        self.labelUpdate_onQueueChange()



    def stopSCTRLQueue(self):
        """ set UI indicators, turn of threads """

        self.sendQueueThread.quit()
        self.sendQueueThread.wait()
        css = "border-radius: 20px; \
               background-color: #4c4a48;"
        self.SCTRL_indi_qProcessing.setStyleSheet(css)
        self.DC_indi_robotMoving.setStyleSheet(css)
        self.TCP_indi_qProcessing.setStyleSheet(css)
        self.labelUpdate_onQueueChange()

    

    def addGcodeSgl(self,atID = False):
        """ function meant to convert any single gcode lines to QEntry, gets them only from SGLC frame, yet """

        txt = self.SGLC_entry_gcodeSglComm.toPlainText()       
        
        command,dummy = reShort('^G\d+', txt, 0)
        command = int(command)
        pos = copy.deepcopy(STT_pos)
        speed = copy.deepcopy(DC_speed)

        match command:

            case 1:
                entry = gcodeG1ToQEntry(pos,speed,IO_zone,txt)

            case 28:
                zero = copy.deepcopy(DC_curr_zero)
                entry = QEntry(ID=0, COOR_1=zero, SV=speed, Z=IO_zone)
            
            case 92:
                # not ready, needs to distinguish between G92 for X, Y and Z or G92 for E
                # mutex.lock()
                # DC_curr_zero = pos
                # mutex.unlock()
                return None
            
            case _:
                self.SGLC_entry_gcodeSglComm.setText("SYNTAX ERROR: \n" + txt)
                return None

        
        if(atID):
            entry.ID = int(self.SGLC_num_gcodeSglComm_addByID.value())

        mutex.lock()
        res = SC_queue.add(entry)
        mutex.unlock()
        
        if(res == ValueError):
            self.SGLC_entry_gcodeSglComm.setText("VALUE ERROR: \n" + txt)
            return None
        
        self.labelUpdate_onQueueChange()
        return entry

    

    def addRapidSgl(self,atID = False):
        """ function meant to convert all RAPID single lines into QEntry, only SGLC frame, yet """

        txt = self.SGLC_entry_rapidSglComm.toPlainText() 
        
        pos = copy.deepcopy(STT_pos)
        entry = QEntry(ID=-1)
        entry,err = rapidToQEntry(txt)

        if( entry == None ):
            print(err)
            self.SGLC_entry_rapidSglComm.setText("SYNTAX ERROR: \n" + txt)
            return None
        
        if(atID):
            entry.ID = self.SGLC_num_rapidSglComm_addByID.value()

        mutex.lock()
        res = SC_queue.add(entry)
        mutex.unlock()

        if(res == ValueError):
            self.SGLC_entry_rapidSglComm.setText("VALUE ERROR: \n" + txt)
            return None

        self.labelUpdate_onQueueChange()
        return entry



    #####################################################################################################
    #                                          DC COMMANDS                                              #
    #####################################################################################################

    def homeCommand(self):
        """ sets up a command to drive back to DC_curr_zero, gives it to the actual sendCommand function """

        zero = copy.deepcopy(DC_curr_zero)

        readMT = self.DC_drpd_moveType.currentText()
        if(readMT == "JOINT"):
            mt = 'J'
        else:
            mt = 'L'

        command = QEntry(ID=SC_curr_comm_id, MT=mt, COOR_1=zero, SV=DC_speed, Z=0)
        return self.sendCommand(command, DC = True)
        


    def sendDCCommand(self, axis = '0', dir = '+'):
        """ sets up a command accourding to the DC frames input, gives it to the actual sendCommand function """

        stepWidth = self.DC_sld_stepWidth.value()
        match stepWidth:
            case 1:
                pass
            case 2:
                stepWidth = 10
            case 3:
                stepWidth = 100
            case _:
                return ValueError

        if(dir != '+' and dir != '-'):
            return ValueError
        if(dir == '-'):
            stepWidth = -stepWidth

        mutex.lock
        newPos = copy.deepcopy(STT_pos)
        mutex.unlock

        match axis:
            case 'X':
                newPos.X += stepWidth
            case 'Y':
                newPos.Y += stepWidth
            case 'Z':
                newPos.Z += stepWidth
            case 'EXT':
                newPos.EXT += stepWidth
            case _:
                return ValueError
            
        
        readMT = self.DC_drpd_moveType.currentText()
        if(readMT == "JOINT"):
            mt = 'J'
        else:
            mt = 'L'

        command = QEntry(ID=SC_curr_comm_id, MT=mt, COOR_1=newPos, SV=DC_speed, Z=0)
        return self.sendCommand(command, DC = True)



    def sendNCCommand(self,axis):
        """ sets up a command according to NC absolute positioning, gives it to the actual sendCommand function """

        newPos = copy.deepcopy(STT_pos)
        lineEdits = {1: self.NC_entry_x,
                     2: self.NC_entry_y,
                     3: self.NC_entry_z,
                     4: self.NC_entry_xOrient,
                     5: self.NC_entry_yOrient,
                     6: self.NC_entry_zOrient,
                     # 7 is a placeholder for Q, which can not be set by hand
                     8: self.NC_entry_ext}
        valueLst = {}
        err = False
        
        for n in axis:
            res = txtEntryToFloat(lineEdits[n].text())
            if( not (type(res) is float) ):
                lineEdits[n].setText(res)
                err = True
            else:
                valueLst[n] = res

        if (err):
            return None
        
        if 1 in valueLst:
            newPos.X = valueLst[1]
        if 2 in valueLst:
            newPos.Y = valueLst[2]
        if 3 in valueLst:
            newPos.Z = valueLst[3]
        if 4 in valueLst:
            newPos.X_ori = valueLst[4]
        if 5 in valueLst:
            newPos.Y_ori = valueLst[5]
        if 6 in valueLst:
            newPos.Z_ori = valueLst[6]
        if 8 in valueLst:
            newPos.EXT = valueLst[8]

        readMT = self.DC_drpd_moveType.currentText()
        if(readMT == "JOINT"):
            mt = 'J'
        else:
            mt = 'L'

        command = QEntry(ID=SC_curr_comm_id, MT=mt, COOR_1=newPos, SV=DC_speed, Z=0)
        return self.sendCommand(command, DC = True)
    


    def sysCommand(self,type = ''):
        """ sets up non-moving-type commands, gives it to the actual sendCommand function
            not ready, fix E case """

        if ( (type != 'S') and (type != 'E') ):
            return KeyError
        command = QEntry(ID=SC_curr_comm_id,MT=type)
        return self.sendCommand(command, DC = True)


    
    def sendCommand(self,command, DC = False):
        """ actual sendCommand function, uses the TCPIP class from utilies, handles errors (not done yet) """

        msg, msgLen = ROBO_tcpip.send(command)
        
        if (msg == True):
            global SC_curr_comm_id
            
            if(DC):
                mutex.lock()
                SC_queue.increment()
                SC_curr_comm_id = SC_curr_comm_id + 1
                mutex.unlock()

            self.TCP_ROB_disp_writeBuffer.setText(str(command))
            self.TCP_ROB_disp_bytesWritten.setText(str(msgLen))
            return msg

        elif (msg == ValueError):
            self.logEntry('CONN','TCPIP class "ROBO_tcpip" encountered ValueError in sendCommand, data length: ' + str(msgLen))
            self.TCP_ROB_disp_writeBuffer.setText('ValueError')
            self.TCP_ROB_disp_bytesWritten.setText(str(msgLen))
        
        elif (msg == RuntimeError or msg == OSError):
            self.logEntry('CONN','TCPIP class "ROBO_tcpip" encountered RuntimeError/OSError in sendCommand..')
            self.TCP_ROB_disp_writeBuffer.setText('RuntimeError/OSError')
            self.TCP_ROB_disp_bytesWritten.setText(str(msgLen))
        
        else:
            self.logEntry('CONN','TCPIP class "ROBO_tcpip" encountered ' + str(msg))
            self.TCP_ROB_disp_writeBuffer.setText('unspecified error')
            self.TCP_ROB_disp_bytesWritten.setText(str(msgLen))
    
        return msg


    def setZero(self,axis):
        """ overwrite DC_curr_zero, uses deepcopy to avoid mutual large mutual exclusion blocks """

        newZero = copy.deepcopy(DC_curr_zero)
        currPos = copy.deepcopy(STT_pos)

        if 1 in axis:
            newZero.X = currPos.X
        if 2 in axis:
            newZero.Y = currPos.Y
        if 3 in axis:
            newZero.Z = currPos.Z
        if 4 in axis:
            newZero.X_ori = currPos.X_ori
        if 5 in axis:
            newZero.Y_ori = currPos.Y_ori
        if 6 in axis:
            newZero.Z_ori = currPos.Z_ori
        # 7 is a placeholder for Q, which can not be set by hand
        if 8 in axis:
            newZero.EXT = currPos.EXT
        
        mutex.lock()
        DC_curr_zero = newZero
        mutex.unlock()



    
    #####################################################################################################
    #                                           CLOSE UI                                                #
    #####################################################################################################

    def closeEvent(self, event):
        """ exit all threads and connections clean(ish) """

        self.logEntry('GNRL','closeEvent signal.')
        self.logEntry('GNRL','end threading, delete Threads...')
        
        if(ROBO_tcpip.connected):
            self.logEntry('CONN','closing TCP connections...')
            self.sysCommand('E')

            self.receiveThread.quit()
            self.receiveThread.wait()
            ROBO_tcpip.close()

        self.receiveThread.deleteLater()

        self.logEntry('GNRL','exiting GUI.')
        event.accept()





####################################################   MAIN  ####################################################

# define and create a logfile, which carries its creation datetime in the title
logpath = Path(r"C:\Users\\Max\\Desktop\\PRINT_log")
logpath.mkdir(parents=True,exist_ok=True)
logpath = logpath / Path(str(datetime.now().strftime('%Y-%m-%d_%H%M%S')) + ".txt")
text = str(datetime.now().strftime('%Y-%m-%d_%H%M%S ')) + "GNRL: program booting, starting GUI...\n"
logfile = open(logpath,'x')
logfile.write(text)
logfile.close()


# overwrite ROBO_tcpip for testing, delete later
ROBO_tcpip.IP = 'localhost'
ROBO_tcpip.PORT = 10001
ROBO_tcpip.C_TOUT = 30.0
ROBO_tcpip.R_TOUT = 1
ROBO_tcpip.W_TOUT = 1
ROBO_tcpip.R_BL = 36
ROBO_tcpip.W_BL = 159


# mutual exclusion object, used to manage global data exchange
mutex = QMutex()

# start the UI and show the window to user
app = 0                             # leave that here so app doesnt include the remnant of a previous QApplication instance
win = 0
app = QApplication(sys.argv)
win = Mainframe()
win.show()
app.exec()
# sys.exit(app.exec())

99.0, 88, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0
RECEIVED: b'\x00\x00\xc6BX\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@' X1.0, Y2.0, Z3.0, X_ori4.0, Y_ori5.0, Z_ori6.0, Q0, EXT7.0
99.0, 88, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0
RECEIVED: b'\x00\x00\xc6BX\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@' X1.0, Y2.0, Z3.0, X_ori4.0, Y_ori5.0, Z_ori6.0, Q0, EXT7.0
99.0, 88, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0
RECEIVED: b'\x00\x00\xc6BX\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@' X1.0, Y2.0, Z3.0, X_ori4.0, Y_ori5.0, Z_ori6.0, Q0, EXT7.0
99.0, 88, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0
RECEIVED: b'\x00\x00\xc6BX\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@' X1.0, Y2.0, Z3.0, X_ori4.0, Y_ori5.0, Z_ori6.0, Q0, EXT7.0
99.0, 88, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0
RECEIVED: b'\x00\x00\xc6BX\x00\x

0

In [2]:
ROBO_tcpip.IP = 'localhost'
ROBO_tcpip.PORT = 10001
ROBO_tcpip.C_TOUT = 30.0
ROBO_tcpip.R_TOUT = 30
ROBO_tcpip.W_TOUT = 30
ROBO_tcpip.R_BL = 36
ROBO_tcpip.W_BL = 159

print(ROBO_tcpip.connect())

(True, ('localhost', 10001))


In [3]:
entry = QEntry()
ans = RoboTelemetry()

ROBO_tcpip.send(entry)
ans = ROBO_tcpip.receive()

In [4]:
ROBO_tcpip.close()

In [11]:
IP,PORT,TIO = '192.168.125.1',4196,60

class PwrUp(QDialog, Ui_Dialog):
    def __init__(self,parent=None):
        super().__init__(parent)
        self.setupUi(self)
        self.label.setText('Blibla ' + IP + ', ' + str(PORT))
        self.connectMainSignals()

    def connectMainSignals(self):
        pass
        # self.accepted.connect(self.accept)
        # self.rejected.connect(self.reject)
        # self.action_Exit.triggered.connect(self.close)


res = 0
if __name__ == '__main__':
    app = 0                             # leave that here so app doesnt include the remnant of a previous QApplication instance
    app = QApplication(sys.argv)
    win = PwrUp()
    win.show()
    app.exec()
    res = win.result()
    # sys.exit(app.exec())
else:
    print('Nope.')

if(res):
    print('User yes')
else:
    print('User NO')

User yes


In [21]:
key = b'\x00\x00\x00\x00LE\x00\x00@@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x002\x00\x00\x002\x00\x00\x00\xc8\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00V\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'.decode('utf-8', errors='ignore')
string = ""
for i in key:
    string += i
string

'\x00\x00\x00\x00LE\x00\x00@@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x002\x00\x00\x002\x00\x00\x00\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00V\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [43]:
Path.joinpath(Path(r'C:\\Users\\Max\\Desktop'), Path(str(datetime.now().strftime('%Y-%m-%d_%H%M%S')) + '.txt'))

None
